<a href="https://colab.research.google.com/github/mtuan8820/learnNLP/blob/main/Poetry_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt


File ‘robert_frost.txt’ already there; not retrieving.



In [2]:
import numpy as np
import string

In [3]:
input_texts = []
for line in open('robert_frost.txt'):
  line = line.rstrip().lower()
  if line:
    line = line.translate(str.maketrans('','',string.punctuation))
    input_texts.append(line)


In [4]:
idx = 1
word2idx = {}

In [5]:
for text in input_texts:
  tokens = text.split()
  for token in tokens:
    if token not in word2idx:
      word2idx[token] = idx
      idx+=1

In [6]:
text_int = []
for text in input_texts:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  text_int.append(line_as_int)

In [7]:
A1 = {}
pi = {}
A2 = {}

In [11]:
def generate_word_prob_dict(text_int,pi, A1, A2):
  for token in text_int:
    prev_prev_idx = None
    prev_idx = None
    for idx in token:
        if prev_prev_idx is None and prev_idx is None:
            # update pi matrix
            if idx not in pi:
                pi[idx] = 1
            else:
                pi[idx] +=1
        elif prev_prev_idx is None and prev_idx is not None:
            # update A1 matrix
            # pdb.set_trace()
            if A1 == {} or prev_idx not in A1:
                A1[prev_idx] = {idx : 1}
            elif idx not in A1[prev_idx]:
                A1[prev_idx] = {idx : 1}
            else:
                A1[prev_idx][idx] +=1
        else:
            #update A2 matrix
            if A2 == {} or (prev_prev_idx, prev_idx) not in A2:
                A2[(prev_prev_idx, prev_idx)] = {idx : 1}
            if idx not in A2[(prev_prev_idx, prev_idx)]:
                A2[(prev_prev_idx, prev_idx)][idx] = 1
            else:
                A2[(prev_prev_idx, prev_idx)][idx] +=1
        prev_prev_idx = prev_idx
        prev_idx = idx
  return pi, A1, A2

In [15]:
pi, A1, A2 =  generate_word_prob_dict(text_int, pi, A1, A2)
list(A2.items())[:5]

[((1, 2), {3: 11}),
 ((2, 3), {4: 11}),
 ((3, 4), {5: 11}),
 ((4, 5),
  {6: 6,
   7: 5,
   986: 5,
   1026: 5,
   1424: 5,
   1042: 5,
   1410: 5,
   1714: 5,
   244: 5,
   692: 5,
   1001: 5,
   1924: 5,
   1019: 10}),
 ((5, 6), {7: 6})]

In [20]:
for key in A1:
  s = sum(A1[key].values())
  for subkey in A1[key]:
    A1[key][subkey] = A1[key][subkey]/s

for key in A2:
  s = sum(A2[key].values())
  for subkey in A2[key]:
    A2[key][subkey] = A2[key][subkey]/s

s = sum(pi.values())
for key in pi:
  pi[key] = pi[key]/s


In [24]:
list(pi.items())[:5]

[(1, 0.005571030640668524),
 (8, 0.08983286908077995),
 (24, 0.034818941504178275),
 (30, 0.008356545961002786),
 (39, 0.0006963788300835655)]

In [26]:
# prompt:  create class TextGenerator using markov model

class TextGenerator:
  def __init__(self, pi, A1, A2, word2idx):
    self.pi = pi
    self.A1 = A1
    self.A2 = A2
    self.word2idx = word2idx
    self.idx2word = {v: k for k, v in word2idx.items()}

  def generate_text(self, length=100):
    # Generate text of given length
    prev_prev_idx = None
    prev_idx = None
    text = []
    for i in range(length):
      if prev_prev_idx is None and prev_idx is None:
        # Choose the first word based on pi
        idx = self.sample_from_probs(self.pi)
      elif prev_prev_idx is None and prev_idx is not None:
        # Choose the next word based on A1
        idx = self.sample_from_probs(self.A1[prev_idx])
      else:
        # Choose the next word based on A2
        idx = self.sample_from_probs(self.A2[(prev_prev_idx, prev_idx)])
      text.append(self.idx2word[idx])
      prev_prev_idx = prev_idx
      prev_idx = idx
    return " ".join(text)

  def sample_from_probs(self, probs):
    # Sample a word based on probabilities
    total = sum(probs.values())
    rand = np.random.rand() * total
    cumulative = 0
    for idx, prob in probs.items():
      cumulative += prob
      if cumulative > rand:
        return idx
    return list(probs.keys())[-1]  # Return the last key if no match found


In [35]:
generator = TextGenerator(pi, A1, A2, word2idx)
text = generator.generate_text(100)
text

KeyError: (2046, 45)